In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, Conv2D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import nltk
import string
from string import digits
import numpy as np
import os
import pandas as pd
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
from collections import Counter
from nltk.stem import SnowballStemmer
from string import punctuation
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import pickle

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import ktrain
from ktrain import text
import pandas as pd
import re
import string
from string import digits
import numpy as np

In [ ]:
#Loading the dataset and assigning labels

data=pd.read_csv('/content/a.txt',sep="\n",header=None,names=['Examples'])
labels=[0]*data.size
data.insert(1,"label",labels)
data1=pd.read_csv('/content/q.txt',sep="\n",header=None,names=['Examples'])
labels=[1]*data1.size
data1.insert(1,"label",labels)
data=data.append(data1)
data['Examples']=data['Examples'].astype(str)

In [ ]:
#Data cleaning

def clean_text(text):
    #print(text)
    ## Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    translator = str.maketrans('','',digits)
    text=text.translate(translator)
    
    ## Convert words to lower case and split them
    text = text.lower()
    
    
    ## Remove stop words
    #stops = set(stopwords.words("english"))
    #text = [w for w in text if not w in stops and len(w) >= 3]
    
    #text = "\n".join([ll.rstrip() for ll in text.splitlines() if ll.strip()])
    #text = " ".join(text)
    #text.replaceAll("\\d","")
    #text = filter(lambda x: not re.match(r'^\s*$', x), text)
    """
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"\n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", "", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text) 
    """
    ## Stemming
    
    #text = text.split()
    #stemmer = SnowballStemmer('english')
    #stemmed_words = [stemmer.stem(word) for word in text]
    #text = " ".join(stemmed_words)
    
    text=re.sub(r"\b[a-zA-Z]\b", "", text)

    return text

In [ ]:
#More data pre-processing and removing empty sentences

data['Examples'] = data['Examples'].map(lambda x: clean_text(x))
data['Examples'].replace('',np.nan,inplace=True)
data=data.dropna()
data['Examples']=data['Examples'].astype(str)

In [ ]:
#for using ktrain library to use bert

data['Statement']=data['label']
for i in range(len(data)) :
    if data['label'].iloc[i]==0:
        data['Statement'].iloc[i]=1
    else:
        data['Statement'].iloc[i]=0


data=data.rename(columns={'label':'Question'})

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
#Bert model using ktrain library
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(data, 
                                                                   'Examples', # name of column containing review text
                                                                   label_columns=['Question', 'Statement'],
                                                                   maxlen=75, 
                                                                   max_features=100000,
                                                                   preprocess_mode='bert',
                                                                   val_pct=0.33,
                                                                   ngram_range=3)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier('bert', (x_train, y_train) , preproc=preproc)
learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=32)

#learner.lr_find(show_plot=True)

Is Multi-Label? False
maxlen is 75
done.


In [ ]:
model = learner.load_model('/content/drive/My Drive/SentenceClassification/')

In [ ]:
learner.validate()

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     43073
           1       1.00      1.00      1.00     37410

    accuracy                           1.00     80483
   macro avg       1.00      1.00      1.00     80483
weighted avg       1.00      1.00      1.00     80483



array([[42971,   102],
       [   80, 37330]])

In [ ]:
learner.fit_onecycle(1e-6, 1)



begin training using onecycle policy with max lr of 1e-06...
Train on 163404 samples, validate on 80483 samples
163404/163404 [==============================] - 4118s 25ms/sample - loss: 0.0235 - accuracy: 0.9949 - val_loss: 0.0099 - val_accuracy: 0.9975


In [ ]:
learner.save_model('/content/')

In [ ]:
learner.view_top_losses(n=1, preproc=preproc)


In [ ]:
#dataP has sentences which are ambiguous for the model to predict. The test set has sentences similar to the train set hence to test the quality of classification of the model, these sentences are tested.

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/Glove')
dataP = [
        'This can be done by us',
        'This is not what is expected from you',
        'Dont do this',
        'This is a statement',
        'Predict this as a statement',
        'I am a guy',
        'What is you name',
        'Why cant you do this',
        'This is why I cant do this',
         'This is not how you should behave',
         'Why I cant do this',
         'Why cant I do this',
         'Why should you be doing this',
         'This is why you should be doing this'
         ]

#here, Question => Statement, Statement => Question (Labels were interchanged while training BERT model)
predictor.predict(dataP)

['Question',
 'Question',
 'Question',
 'Question',
 'Question',
 'Question',
 'Statement',
 'Statement',
 'Question',
 'Question',
 'Question',
 'Statement',
 'Statement',
 'Question']

In [ ]:
#using count vectorizer to convert words to vectors

vectorizer = CountVectorizer(max_features=20000).fit(data['Examples'])
X = vectorizer.transform(data['Examples'])
Y=data.iloc[:,1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
print(X_train.shape)
#X_train=X_train.toarray()
#X_test=X_test.toarray()
#To count the frequency of each word in the corpus for deciding the vocabulary depending on the counts
'''
sum_words=X.sum(axis=0)
vectorizer.vocabulary_.items()
words_freq = [(word, sum_words[0, idx]) for word, idx in     vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[19000:20000]
'''


(163404, 20000)


'\nsum_words=X.sum(axis=0)\nvectorizer.vocabulary_.items()\nwords_freq = [(word, sum_words[0, idx]) for word, idx in     vectorizer.vocabulary_.items()]\nwords_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)\nwords_freq[19000:20000]\n'

In [ ]:
#Using tf-idf for converting words to vectors

Tfidf_vect = TfidfVectorizer(max_features=20000)
Tfidf_vect.fit(data['Examples'])
X=data.iloc[:,0]
Y=data.iloc[:,1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
print(X_train.shape)
X_train = Tfidf_vect.transform(X_train)
X_test = Tfidf_vect.transform(X_test)
print(X_train.shape)

(163404,)
(163404, 20000)


In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(X_train,Y_train)
# predict the labels on test dataset
predictions_NB = Naive.predict(X_test)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",precision_score(predictions_NB, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",recall_score(predictions_NB, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",f1_score(predictions_NB, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",roc_auc_score(predictions_NB, Y_test*100))
print(confusion_matrix(predictions_NB, Y_test))
tn, fp, fn, tp = confusion_matrix(predictions_NB, Y_test).ravel()
tn,fp,fn,tp

Naive Bayes Accuracy Score ->  84.20536013816582
Naive Bayes Accuracy Score ->  98.27271677635378
Naive Bayes Accuracy Score ->  78.05616184589425
Naive Bayes Accuracy Score ->  87.00549956044405
Naive Bayes Accuracy Score ->  0.8758751394043421
[[25214   748]
 [11964 42557]]


(25214, 748, 11964, 42557)

In [ ]:
#Training using random Forest algorithm


clf = RandomForestClassifier(max_depth=15, random_state=0)
clf.fit(X_train,Y_train)
clf.score(X_test,Y_test)
predictions_RF=clf.predict(X_test)

print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_RF, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",precision_score(predictions_RF, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",recall_score(predictions_RF, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",f1_score(predictions_RF, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",roc_auc_score(predictions_RF, Y_test)*100)
print(confusion_matrix(predictions_RF, Y_test))

Naive Bayes Accuracy Score ->  95.25365605158854
Naive Bayes Accuracy Score ->  97.36750952545896
Naive Bayes Accuracy Score ->  94.02385996209165
Naive Bayes Accuracy Score ->  95.66647759500852
Naive Bayes Accuracy Score ->  95.41251362771511
[[34498  1140]
 [ 2680 42165]]


In [ ]:
# Classifier Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train,Y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_SVM, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",precision_score(predictions_SVM, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",recall_score(predictions_SVM, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",f1_score(predictions_SVM, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",roc_auc_score(predictions_SVM, Y_test)*100)
print(confusion_matrix(predictions_SVM, Y_test))

SVM Accuracy Score ->  98.98487879427954
Naive Bayes Accuracy Score ->  98.98487879427954
Naive Bayes Accuracy Score ->  98.90081976677058
Naive Bayes Accuracy Score ->  99.21009960620802
Naive Bayes Accuracy Score ->  99.05521827117663
Naive Bayes Accuracy Score ->  98.96720240407417
[[36837   476]
 [  341 42829]]


In [ ]:
#Using CNN with tf-idf/count vectorizers

model_conv = Sequential()
'''
model_conv.add(Conv1D(64, 5, activation='relu',input_shape=(20000,1)))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(100))
'''
model_conv.add(Dense(100, activation='relu',input_shape=(20000,)))
model_conv.add(Dense(1, activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_conv.summary()
model_conv.fit(X_train, Y_train, validation_split=0.2, epochs = 5)
model_conv.evaluate(X_test,Y_test)

predictions_CNN = model_conv.predict(X_test)
print("SVM Accuracy Score -> ",accuracy_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",precision_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",recall_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",f1_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",roc_auc_score(predictions_CNN, Y_test)*100)
print(confusion_matrix(predictions_CNN, Y_test))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               2000100   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 2,000,201
Trainable params: 2,000,201
Non-trainable params: 0
_________________________________________________________________
Train on 130723 samples, validate on 32681 samples
Epoch 1/5
130723/130723 [==============================] - 39s 296us/step - loss: 0.0788 - accuracy: 0.9770 - val_loss: 0.0408 - val_accuracy: 0.9881
Epoch 2/5
130723/130723 [==============================] - 37s 283us/step - loss: 0.0231 - accuracy: 0.9934 - val_loss: 0.0404 - val_accuracy: 0.9882
Epoch 3/5
130723/130723 [==============================] - 37s 282us/step - loss: 0.0140 - accuracy: 0.9959 - val_loss: 0.0441 - val_accuracy: 0.9877
Epoch

NameError: ignored

In [ ]:
predictions_CNN = model_conv.predict_classes(X_test)
print("SVM Accuracy Score -> ",accuracy_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",precision_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",recall_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",f1_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",roc_auc_score(predictions_CNN, Y_test)*100)
print(confusion_matrix(predictions_CNN, Y_test))

SVM Accuracy Score ->  98.81962650497621
Naive Bayes Accuracy Score ->  98.81962650497621
Naive Bayes Accuracy Score ->  99.05784551437479
Naive Bayes Accuracy Score ->  98.75227330279243
Naive Bayes Accuracy Score ->  98.90482338836117
Naive Bayes Accuracy Score ->  98.8254401823324
[[36636   408]
 [  542 42897]]


In [ ]:
sent=[
        'This can be done by us',
        'This is not what is expected from you',
        'Dont do this',
        'This is a statement',
        'Predict this as a statement',
        'I am a guy',
        'What is you name',
        'Why cant you do this',
        'This is why I cant do this',
         'This is not how you should behave',
         'Why I cant do this',
         'Why cant I do this',
         'Why should you be doing this',
         'This is why you should be doing this'
         ]
ans=Tfidf_vect.transform(sent)
print(clf.predict(ans))
print(SVM.predict(ans))
print(Naive.predict(ans))
print(model_conv.predict_classes(ans))


[1 1 0 0 0 0 1 1 1 1 1 1 1 1]
[0 1 1 0 0 0 1 1 1 1 1 1 1 1]
[1 1 1 1 1 0 1 1 1 1 1 1 1 1]
[[0]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


ValueError: ignored

In [ ]:
#Saving models

filename = '/content/drive/My Drive/Glove/SVMmodel.sav'
pickle.dump(SVM, open(filename, 'wb')) 

filename = '/content/drive/My Drive/Glove/NaiveBayesmodel.sav'
pickle.dump(Naive, open(filename, 'wb')) 

filename = '/content/drive/My Drive/Glove/CNNmodel.sav'
pickle.dump(model_conv, open(filename, 'wb')) 


filename = '/content/drive/My Drive/Glove/RandomForestmodel.sav'
pickle.dump(clf, open(filename, 'wb')) 

In [ ]:
vocabulary_size = 30000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data['Examples'])
len(tokenizer.word_index)+ 1

63557

In [ ]:
#For using pre-trained word embeddings (Preparing embedding Matrix)
test=list()
embeddings_index = dict()
f = open('/content/drive/My Drive/SentenceClassification/glove.6B.300d.txt')
for line in f:
    values = line.split()
    test.append(values[0])
    words = values[0]
    if(words == 'destiny'):
        print('present')
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[words] = coefs
f.close()

embedding_matrix = np.zeros((vocabulary_size, 300))
count=0
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
        else:
            count=count+1

present


In [ ]:
#Tokenising and data splitting

X=data.iloc[:,0]
Y=data.iloc[:,1]
sequences = tokenizer.texts_to_sequences(X)
X = pad_sequences(sequences, maxlen=50)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
X_train.shape

(163404, 50)

In [ ]:
#Model that uses keras's embedding layer directly

model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(X_train, Y_train, validation_split=0.2, epochs=5,batch_size=128)
model.evaluate(X_test,Y_test)

predictions_CNN=model.predict_classes(X_test)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 100)           2000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 2,080,501
Trainable params: 2,080,501
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 130723 samples, validate on 32681 samples
Epoch 1/5
130723/130723 [==============================] - 175s 1ms/step - loss: 0.0628 - accuracy: 0.9786 - val_loss: 0.0293 - val_accuracy: 0.9910
Epoch 2/5
130723/130723 [==============================] - 174s 1ms/step - loss: 0.0208 - accuracy: 0.9939 - val_loss: 0.0287 - val_accuracy: 0.9915
Epoch 3/5
130723/130723 [==============================] - 175s 1ms/step - loss: 0.0121 - accuracy: 0.9965 - val_loss: 0.0285 - val_accuracy: 0.9912
Epoch 4/5
130723/130723 [==============================] - 174s 1ms/step - loss: 0.0079 - accuracy: 0.9976 - val_loss: 0.0377 - val_accuracy: 0.9909
Epoch 5/5
80483/80483 [==============================] - 31s 391us/step


In [ ]:
#Model with a convolution layer ahead of LSTM layer inorder to reduce training time by using filters in CNN

model_conv = Sequential()
model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(100))
model_conv.add(Dense(1, activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
model_conv.fit(X_train, Y_train, validation_split=0.2, epochs = 3)
model_conv.evaluate(X_test,Y_test)

predictions_CNN=model_conv.predict(X_test)

In [ ]:
#Model using pre-trained word embeddings

model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 300, input_length=50, weights=[embedding_matrix], trainable=False))
'''
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
'''
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_glove.summary()
model_glove.fit(X_train,Y_train, validation_split=0.2, epochs = 7,batch_size=512)
model_glove.evaluate(X_test,Y_test)

predictions_CNN=model_glove.predict_classes(X_test)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 300)           9000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 9,160,501
Trainable params: 160,501
Non-trainable params: 9,000,000
_________________________________________________________________
Train on 130723 samples, validate on 32681 samples
Epoch 1/7
130723/130723 [==============================] - 25s 190us/step - loss: 0.1159 - accuracy: 0.9544 - val_loss: 0.0422 - val_accuracy: 0.9861
Epoch 2/7
130723/130723 [==============================] - 23s 179us/step - loss: 0.0345 - accuracy: 0.9883 - val_loss: 0.0315 - val_accuracy: 0.9904
Epoch 3/7
130723/

In [ ]:
print("SVM Accuracy Score -> ",accuracy_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",precision_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",recall_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",f1_score(predictions_CNN, Y_test)*100)
print("Naive Bayes Accuracy Score -> ",roc_auc_score(predictions_CNN, Y_test)*100)
print(confusion_matrix(predictions_CNN, Y_test))

SVM Accuracy Score ->  99.32780835704435
Naive Bayes Accuracy Score ->  99.32780835704435
Naive Bayes Accuracy Score ->  99.30723934880498
Naive Bayes Accuracy Score ->  99.44272302640707
Naive Bayes Accuracy Score ->  99.3749350094164
Naive Bayes Accuracy Score ->  99.31853636617237
[[36937   300]
 [  241 43005]]


In [ ]:
sent=[
        'This can be done by us',
        'This is not what is expected from you',
        'Dont do this',
        'This is a statement',
        'Predict this as a statement',
        'I am a guy',
        'What is you name',
        'Why cant you do this',
        'This is why I cant do this',
         'This is not how you should behave',
         'Why I cant do this',
         'Why cant I do this',
         'Why should you be doing this',
         'This is why you should be doing this'
         ]
seq=tokenizer.texts_to_sequences(sent)
test = pad_sequences(seq, maxlen=50)
model_glove.predict_classes(test)

array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1]], dtype=int32)

In [ ]:
filename = '/content/drive/My Drive/SentenceClassification/CNNGloveembeddingmodel.sav'
pickle.dump(model_glove, open(filename, 'wb')) 

In [ ]:
sent=["in the barbie world"]
ans=Tfidf_vect.transform(sent)
Naive.predict(ans)

array([1])